In [1]:
import numpy as np
import agentpy as ap
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import seaborn as sns, IPython
from queue import PriorityQueue

class CityAgent(ap.Agent):
    
    '''
    Start agent with 4 actions
    '''
    def setup(self):
        self.actions = {'up': (-1,0), 'down': (1, 0), 'left': (0, -1), 'right': (0, 1)}
        self.env = self.model.env
        self.steps = 0
        self.i = 0

    '''
    Move agent to next step
    '''
    def execute(self):
        if self.i < len(self.path):
            self.env.move_to(self, self.path[self.i])
            self.steps += city[self.path[self.i]]
            self.i += 1

    '''
    Get manhattan distance for heuristic
    '''
    def get_distance(self, a, b):
        return abs(a[0] - b[0]) + abs(a[1] - b[1])
    
    '''
    Use A* to find optimal solution
    '''
    def solve(self):
        n,m = city.shape

        pq = PriorityQueue()
        pq.put((0,self.p.init))

        scores = {self.p.init: 0}
        curr_path = {}

        while not pq.empty():
            _, current_pos = pq.get()
            
            if current_pos == self.p.goal:
                print("Solved")
                self.path = []
                while current_pos in curr_path:
                    self.path.append(current_pos)
                    current_pos = curr_path[current_pos]
                self.path = self.path[::-1]
                print("Path: ", self.path)
                return
            
            x,y = current_pos
            for _, (dx,dy) in self.actions.items():
                nx,ny = x + dx, y + dy
                if 0 <= nx < n and 0 <= ny < m and city[nx][ny] != -1 and city[nx][ny] != -10:
                    tentative_g_score = scores[current_pos] + city[nx][ny]

                    if (nx, ny) not in scores or tentative_g_score < scores[(nx, ny)]:
                        scores[(nx, ny)] = tentative_g_score
                        f_score = tentative_g_score + self.get_distance((nx, ny), self.p.goal)
                        pq.put((f_score, (nx, ny)))  
                        curr_path[(nx, ny)] = current_pos

    '''
    Get position of the agent
    '''
    def get_position(self):
        return self.env.positions[self]

class City(ap.Grid):
    '''
    Start city
    '''
    def setup(self):
        self.city = self.p.city

class CityModel(ap.Model):
    
    '''
    Start model
    '''
    def setup(self):
        city[self.p.goal] = 1
        self.env = City(self, shape=city.shape)
        self.agent = CityAgent(self)
        self.env.add_agents([self.agent], positions=[self.p.init])
        self.agent.solve()

    '''
    On each step, agent moves
    '''
    def step(self):
        self.agent.execute()

    '''
    Stop simulation if goal is reached
    '''
    def update(self):
        if self.agent.get_position() == self.model.p.goal:
            self.stop()

'''
Animation
'''
def animation_plot(model, ax):
    n, m = model.p.city.shape
    grid = np.copy(city)
    grid[model.p.goal] = 200

    color_dict = {1: '#ffffff', 2: '#7c4700', 4: '#2a9dfb', 5: '#6e5c56', -1: '#000000', -10: '#4f4f4f', 100: '#0000ff', 200: '#00ff00'}
    ap.gridplot(grid, ax=ax, color_dict=color_dict, convert=True)
    agent = list(model.env.agents)[0]
    grid[model.env.positions[agent]] = 100
    ap.gridplot(grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title("City Agent\nTime: {}".format(agent.steps))

city = np.load('streets-1.npy')

parameters = {
    'city': city,
    'init': (5, 3),
    'goal': (16,26),
    'steps': 100
}

fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111)
mazeModel = CityModel(parameters)
animation = ap.animate(mazeModel, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml())

Solved
Path:  [(5, 2), (5, 1), (5, 0), (6, 0), (7, 0), (8, 0), (9, 0), (10, 0), (11, 0), (12, 0), (13, 0), (14, 0), (15, 0), (16, 0), (17, 0), (18, 0), (19, 0), (20, 0), (20, 1), (20, 2), (20, 3), (20, 4), (20, 5), (20, 6), (20, 7), (20, 8), (20, 9), (20, 10), (20, 11), (20, 12), (20, 13), (20, 14), (20, 15), (20, 16), (20, 17), (20, 18), (20, 19), (20, 20), (20, 21), (20, 22), (20, 23), (20, 24), (20, 25), (19, 25), (18, 25), (17, 25), (16, 25), (16, 26)]
